## Analyze GHArchive Data in Parquet files using Spark

Let us analyze GHArchive Data that are created by our Spark Streaming Process.
* Location: /user/{username}/github/streaming/bronze/data/ghactivity**.
* As the files are in Parquet format, we can use `spark.read.parquet` to read these files.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config('spark.sql.warehouse.dir', f'/user/{username}/warehouse'). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Incremental Loads using Spark Structured Streaming'). \
    master('yarn'). \
    getOrCreate()

In [2]:
!hdfs dfs -ls /user/${USER}/github/streaming/bronze/data/ghactivity

Found 2 items
drwxr-xr-x   - itv007304 supergroup          0 2023-07-29 00:22 /user/itv007304/github/streaming/bronze/data/ghactivity/_spark_metadata
drwxr-xr-x   - itv007304 supergroup          0 2023-07-14 12:17 /user/itv007304/github/streaming/bronze/data/ghactivity/created_year=2023


In [3]:
ghactivity = spark. \
    read. \
    parquet(f"/user/{username}/github/streaming/bronze/data/ghactivity")

In [ ]:
ghactivity.printSchema()

In [ ]:
ghactivity.count()

In [ ]:
ghactivity. \
  filter("type = 'CreateEvent' AND payload.ref_type = 'repository'"). \
  count()

We can also register Dataframe as temporary view and analyze the data using Spark SQL.

In [ ]:
ghactivity.createOrReplaceTempView('ghactivity')

In [ ]:
new_repos = spark.sql("""
    SELECT
        repo.id AS repo_id,
        repo.name AS repo_name,
        actor.id AS actor_id,
        actor.login AS actor_login,
        actor.display_login AS actor_display_login,
        payload.ref_type AS ref_type,
        type,
        created_at,
        year(created_at) AS year,
        month(created_at) AS month,
        dayofmonth(created_at) AS day
    FROM ghactivity
    WHERE 
        type = 'CreateEvent'
        AND payload.ref_type = 'repository'
""")

In [ ]:
display(new_repos)

In [ ]:
new_repos.count()

In [ ]:
spark.sql("""
    SELECT 
        count(1)
    FROM ghactivity
    WHERE 
        type = 'CreateEvent'
        AND payload.ref_type = 'repository'    
""").show()

* Get count by date to confirm the counts by date.

In [4]:
from pyspark.sql.functions import to_date

In [5]:
ghactivity. \
  groupby(to_date('created_at')). \
  count(). \
  show()

Py4JJavaError: An error occurred while calling o67.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: ResultStage 8 (showString at NativeMethodAccessorImpl.java:0) has failed the maximum allowable number of times: 4. Most recent failure reason: org.apache.spark.shuffle.FetchFailedException: java.lang.IllegalArgumentException: Unknown message type: 9 	at org.apache.spark.network.shuffle.protocol.BlockTransferMessage$Decoder.fromByteBuffer(BlockTransferMessage.java:71) 	at org.apache.spark.network.shuffle.ExternalShuffleBlockHandler.receive(ExternalShuffleBlockHandler.java:80) 	at org.apache.spark.network.server.TransportRequestHandler.processRpcRequest(TransportRequestHandler.java:180) 	at org.apache.spark.network.server.TransportRequestHandler.handle(TransportRequestHandler.java:103) 	at org.apache.spark.network.server.TransportChannelHandler.channelRead(TransportChannelHandler.java:118) 	at org.spark_project.io.netty.channel.AbstractChannelHandlerContext.

In [ ]:
ghactivity. \
    select('type', 'payload.ref_type'). \
    distinct(). \
    show(100)

In [ ]:
ghactivity.filter("payload.ref_type = 'repository'").count().show()

In [ ]:
ghactivity. \
    filter("type = 'CreateEvent'"). \
    groupby('payload.ref_type'). \
    count(). \
    show()